## 1. 生成候选项集
- 构建大小为1的所有候选项集的集合C1。
- 扫描数据集，判断是否满足最小支持度的要求，构成集合 L1

In [5]:
def loadDataSet():
    return [[1,3,4], [2,3,5], [1,2,3,5],[2,5]]

def createC1(dataSet):
    C1 = set()
    for transaction in dataSet:
        for item in transaction:
            # 集合内元素应为可哈希类型（hashable type）即 不可变类型
            C1.add(frozenset([item]))
    return C1

In [6]:
# ItemSet = frozenset[int] 项集类型注释
def generate_lk_by_ck(D, Ck, minSupport):
    """
    param1 List[List[int]]  D: 数据集（二维列表嵌套），即Transaction的合集
    param2 Set[ItemSet]          Ck: k-频繁项集的候选集合
    param3 float            minSupport: 最小支持度
    
    return1: Set[ItemSet]            Lk: 经过支持度测试的k-频繁项集的集合
    return2: Dict[Lk, float]    supportData: 保存各频繁项的支持度 
    """
    Lk = set()
    supportData = {} #用于保存各频繁项的支持度
    N = len(D) # 总事务数，作为支持度计算的分母
    # TODO
    # 遍历 Ck, 判断是否出现在各个transaction中
    for ck in Ck:
        # 记录当前候选组合的支持度
        nk = 0
        for trans in D:
            if ck.issubset(trans): nk += 1
        # 计算当前项支持度
        supportk = nk / N
        # 大于最小支持度，选入频繁项
        if supportk >= minSupport:
            Lk.add(ck)
            supportData[ck] = supportk

    return Lk, supportData
            

In [7]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [8]:
C1 = createC1(dataSet)
C1

{frozenset({2}),
 frozenset({3}),
 frozenset({1}),
 frozenset({5}),
 frozenset({4})}

In [9]:
L1, supportData = generate_lk_by_ck(dataSet, C1, 0.5)

## 2. 生成所有频繁项集
- 从 Lk-1 生成 Ck
- 检查候选项子集是否都在频繁 Lk-1 中

In [12]:
from itertools import chain, product, permutations

# def createCk(Lk_1, k):
def createCk(Lk_1, k, L1):#通过频繁项集Lk-1创建Ck候选项集
    """
    Fk-1xF1法生成候选集， 我不得不增加一个传入L1的位置参数
    param1  Set{k-1-ItemSet}   Lk_1: 频繁{k-1}-项集
    param2  int            k   : 指示k-项集长度的整型
    *param3 Set[ItemSet]   L1  : 频繁1-项集
    
    return  Set[k-ItemSet]   Ck: 候选k-项集
    """
    Ck = set()
    # TODO
    prod = product(Lk_1, L1) # 可以遍历
    # 笛卡尔积 p ->（Lk-1, L1)
    for p in prod:
            # 转为形式 frozenset(*Lk-1, *L1), red for redundant
            # why doesn't normal set work
            red_set = frozenset(chain(*p))
            # 仅当长度为k，结果可行
            if len(red_set) == k: Ck.add(red_set) 
    return Ck

def has_infrequent_subset(Ck, Lk_1):#检查候选项Ck的子集是否都在Lk-1中
    # TODO
    for p in permutations(Ck, len(Lk_1)):
        if set(p) in Lk_1 == False: return False
    return True
        

In [3]:
# 生成所有频繁项集
def generate_L(dataSet, minSupport=0.5):
    C1= createC1(dataSet)
    L1, supportData = generate_lk_by_ck(dataSet, C1, minSupport)
    L = [L1]
    k = 2
    Lk_1 = L1.copy()
    while (True):
        Ck = createCk(Lk_1, k, L1)
        # Ck = createCk(Lk_1, k)
        Lk, supK = generate_lk_by_ck(dataSet, Ck, minSupport)
        supportData.update(supK)
        if len(Lk)==0:
            break
        Lk_1 = Lk.copy()
        L.append(Lk_1)
        k += 1
    return L, supportData

In [13]:
L, supportData = generate_L(dataSet, minSupport=0.5)

In [14]:
createCk(L[0], 2, L1)
# createCk(L[0], 2)

{frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({2, 5}),
 frozenset({1, 5}),
 frozenset({3, 5}),
 frozenset({1, 3})}

## 3. 从频繁项集中挖掘关联规则
- 计算规则置信度
- 如果某个规则不满足最小置信度要求，那么该规则的所有子集也不会满足

if $X\rightarrow Y-X$, $X'\rightarrow Y-X'$, $X'\sub X$

In [103]:
def generateRules(dataset, minSupport, minConf=0.7):
    L, supportData=generate_L(dataset,minSupport)
    big_rule = []
    # 2-项集到k-项集
    for i in range(1, len(L)):
        # L[i]: Set[k-ItemSets]
        for freqSet in L[i]:
            # H1 = List[1-ItemSet,]
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:
                rulesFromConseq(freqSet, H1, supportData, big_rule, minConf, L)
            # else:
                # calcConf(freqSet, H1, supportData, big_rule, minConf)
            # 原始版本逻辑如上，会导致{3+}-项集没有后项为1的规则
            calcConf(freqSet, H1, supportData, big_rule, minConf)
    return big_rule

# 新候选规则
def rulesFromConseq(freqSet, H, supportData, big_rule, minConf, L):
    m = len(H[0])
    # freqSet: k-ItemSet
    # len(freqSet) == k, k=1,2,..., max-ItemSet
    if (len(freqSet) > m+1):
        Hmp1 = createCk(H, m+1, L[0])
        _Hmp1 = [h for h in Hmp1 if h.issubset(freqSet)]
        Hmp1 = calcConf(freqSet, _Hmp1, supportData, big_rule, minConf)
        if (len(Hmp1) > 1):
            # 递归调用
            rulesFromConseq(freqSet, Hmp1, supportData, big_rule, minConf, L)

def calcConf(freqSet, H, supportData, big_rule, minConf):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            big_rule.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

蛮力方式，遍历所有子集

In [17]:
def generate_R(dataset, minSupport, minConf):
    L, supportData=generate_L(dataset,minSupport)
    rule_list = []#保存满足置信度的规则
    sub_set_list = []#该数组保存检查过的频繁项
    for i in range(0, len(L)):
        for freq_set in L[i]:
            for sub_set in sub_set_list:#sub_set_list中保存的是L1到Lk-1
                if sub_set.issubset(freq_set):#检查sub_set是否是freq_set的子集
                    #检查置信度是否满足要求，是则添加到规则
                    conf = supportData[freq_set] / supportData[freq_set - sub_set]
                    big_rule = (freq_set - sub_set, sub_set, conf)
                    if conf >= minConf and big_rule not in rule_list:
                        rule_list.append(big_rule)
            sub_set_list.append(freq_set)
    rule_list = sorted(rule_list,key=lambda x:(x[2]),reverse=True)
    return rule_list

In [97]:
# brutal 结果
rules = generate_R(dataSet, 0.5, 0.5)

In [98]:
rules

[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({3, 5}), frozenset({2}), 1.0),
 (frozenset({2, 3}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({2, 5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666)]

In [104]:
# 置信度剪枝结果
rules1 = generateRules(dataSet, 0.5, 0.5)

In [105]:
rules1

[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({3}), frozenset({1}), 0.6666666666666666),
 (frozenset({1}), frozenset({3}), 1.0),
 (frozenset({3}), frozenset({2}), 0.6666666666666666),
 (frozenset({2}), frozenset({3}), 0.6666666666666666),
 (frozenset({5}), frozenset({3}), 0.6666666666666666),
 (frozenset({3}), frozenset({5}), 0.6666666666666666),
 (frozenset({5}), frozenset({2, 3}), 0.6666666666666666),
 (frozenset({3}), frozenset({2, 5}), 0.6666666666666666),
 (frozenset({2}), frozenset({3, 5}), 0.6666666666666666),
 (frozenset({3, 5}), frozenset({2}), 1.0),
 (frozenset({2, 5}), frozenset({3}), 0.6666666666666666),
 (frozenset({2, 3}), frozenset({5}), 1.0)]

In [108]:
# 二者结果相同
set(rules) == set(rules1)

True